In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
from numpy import cos, sin, pi
from scipy import optimize

In [ ]:
# Load data

data = np.loadtxt("Full_16yr_Gamma-ray_LC_TS9_PKS1424-41.txt")

time, flux_obs, flux_err = data[:,0], data[:,1], data[:,2]

# Normalize flux
flux_obs = flux_obs / 1e-6
flux_err = flux_err / 1e-6

In [ ]:
# Modeling the light curve with jet precession model

t0 = time[np.argmax(flux_obs)]
print(f"t0: {t0:.2f}")

alpha = 1.0
p = 2 + alpha

def thetaa(t, Omega, phi0, T, t0):
    Omega = np.radians(Omega)
    phi0 = np.radians(phi0)
    phi = 2 * pi * (t - t0)/ T
    A_t = cos(Omega) * sin(phi0) + sin(Omega) * sin(phi) * cos(phi0)
    B_t = sin(Omega) * cos(phi)
    arg = np.sqrt(A_t**2 + B_t**2)
    arg = np.clip(arg, -1.0, 1.0)
    return np.arcsin(arg)

def doppler(theta, Gamma):
    if Gamma <= 1:
        return np.inf
    beta = np.sqrt(1 - 1 / Gamma**2)
    return 1.0 / (Gamma * (1 - beta * np.cos(theta)))

def model_flux(t, Gamma, Omega, phi0, T, t0, F0):
    th = thetaa(t, Omega, phi0, T, t0)
    delta = doppler(th, Gamma)
    return F0 * delta**p

def log_prior(params):
    Gamma, Omega, phi0, T, t0, F0, log_f = params
    if 1.1 < Gamma < 40 and 20 < Omega < 50 and 1 < phi0 < 10 and 1830 < T < 1910 and 54952 < t0 < 60846 and 0.1 < F0 < 50 and -5 < log_f < 0:
        return 0.0
    return -np.inf

def log_likelihood(params, t, flux, flux_err):
    Gamma, Omega, phi0, T, t0, F0, log_f = params
    model = model_flux(t, Gamma, Omega, phi0, T, t0, F0)
    if np.any(~np.isfinite(model)):
        return -np.inf
    sigma2 = flux_err**2 + model**2 * np.exp(2 * log_f)
    if np.any(sigma2 <= 0):
        return -np.inf
    return -0.5 * np.sum(((flux - model)**2) / sigma2 + np.log(2 * np.pi * sigma2))

def log_posterior(params, t, flux, flux_err):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, t, flux, flux_err)

def chi2(params, t, y, yerr):
    return -2 * log_posterior(params, t, y, yerr)

# Optimization
initial_guess = [15, 20, 5, 1870, 59000, 5, -4]
result = optimize.minimize(
    chi2,
    initial_guess,
    args=(time, flux_obs, flux_err),
    method='Nelder-Mead',
    options={'maxiter': 10000, 'xatol': 1e-8, 'disp': True},
)

if result.success:
    print(" Optimization successful!")
    print("Fitted Parameters:", result.x)
else:
    raise RuntimeError(f"Optimization failed: {result.message}")

# MCMC
pos = result.x * (1 + 1e-6 * np.random.randn(32, 7))
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(time, flux_obs, flux_err))
print(" Running MCMC...")
sampler.run_mcmc(pos, 10000, progress=True)
print(" MCMC finished")

# Analyze samples
burnin = 2000
thin = 15
flat_samples = sampler.get_chain(discard=burnin, thin=thin, flat=True)

# Reduced chi-square evolution
chisquare = []
iteration = []
for i in range(0, len(sampler.get_chain(flat=True)), 1000):
    sample = sampler.get_chain(flat=True)[i]
    Gamma, Omega, phi0, T, t0, F0, log_f = sample
    model = model_flux(time, Gamma, Omega, phi0, T, t0, F0)
    sigma2 = flux_err**2 + model**2 * np.exp(2 * log_f)
    if np.any(sigma2 <= 0) or np.any(~np.isfinite(sigma2)):
        continue
    chi2_val = np.sum((flux_obs - model)**2 / sigma2)
    DOF = len(time) - ndim
    red_chi2 = chi2_val / DOF
    chisquare.append(red_chi2)
    iteration.append(i)


# Plot best-fit model
best_params = np.median(flat_samples, axis=0)
best_model = model_flux(time, *best_params[:6])
plt.errorbar(time, flux_obs, yerr=flux_err, fmt='o', label='Data')
plt.plot(time, best_model, lw=3, label='Best Fit Model', zorder=10)
plt.xlabel('Time')
plt.ylabel('Normalized Flux')
plt.legend()
plt.show()

In [ ]:
burnin = 10
thin = 10
flat_samples = sampler.get_chain(discard=burnin, thin=thin, flat=True)
flat_samples = flat_samples[np.isfinite(flat_samples).all(axis=1)]

import corner

if flat_samples.shape[0] > 1000:
    fig = corner.corner(flat_samples,
                    bins=50,
                   labels = [r"$\Gamma$", r"$\Omega$", r"$\phi_0$", r"$T$", r"$t_0$", r"$F_0$", r"$\log f$"],
                   color='blue',
                   quantiles=[0.16,0.5,0.84],
                   plot_contours=True,
                   fill_contours=True,
                   levels=(0.68,0.95,0.99),
                   plot_datapoints=True,
                   smooth=True,
                   smoothld=True,
                   title_fmt='.4g',
                   show_titles=True,
                   divergences=True, 
                   label_kwargs={"fontsize": 16},
                    title_kwargs={"fontsize": 14}, dpi=500, ls=8)
else:
    print(" Too few valid samples for corner plot")

plt.show()


In [1]:
# Modeling the light curve with jet precession+nutation model

from sympy import *
import lmfit
import emcee

In [ ]:
# -------------------------------------------------------------
# Symbolic Expression Setup for jet precession + nutation model
# -------------------------------------------------------------
def create_symbolic_model():
    Theta_p, OMEGA_p, OMEGA_n, Theta_n = symbols("Theta_p OMEGA_p OMEGA_n Theta_n", positive=True)
    eta_0, Phi_o = symbols("eta_0 Phi_o", real=True)

    # Rotation matrices
    Rz = Matrix([
        [cos(Theta_p), -sin(Theta_p), 0],
        [sin(Theta_p),  cos(Theta_p), 0],
        [0, 0, 1]
    ])
    Ry = Matrix([
        [cos(OMEGA_p), 0, sin(OMEGA_p)],
        [0, 1, 0],
        [-sin(OMEGA_p), 0, cos(OMEGA_p)]
    ])
    Rz_nut = Matrix([
        [cos(eta_0), -sin(eta_0), 0],
        [sin(eta_0),  cos(eta_0), 0],
        [0, 0, 1]
    ])
    Ry_nut = Matrix([
        [cos(Phi_o), 0, sin(Phi_o)],
        [0, 1, 0],
        [-sin(Phi_o), 0, cos(Phi_o)]
    ])

    n_J = Matrix([
        [sin(OMEGA_n)*sin(Theta_n)],
        [sin(OMEGA_n)*cos(Theta_n)],
        [cos(OMEGA_n)]
    ])

    n_s = Rz * Ry * n_J
    n0 = Rz_nut * Ry_nut * n_s
    x, y, z = simplify(n0[0]), simplify(n0[1]), simplify(n0[2])

    phi_expr = asin(sqrt(x**2 + y**2))  # angle between jet and observer

    # Doppler boosting
    Gamma, F0 = symbols("Gamma F0", positive=True)
    doppler = 1 / (Gamma * (1 - sqrt(1 - (1 / Gamma**2)) * cos(phi_expr)))
    model_flux = F0 * doppler**3

    return lambdify(
        (Theta_p, Theta_n, OMEGA_p, OMEGA_n, eta_0, Phi_o, Gamma, F0),
        model_flux, modules='numpy'
    )

model_func = create_symbolic_model()

# -----------------------------------
# Constants and Data
# -----------------------------------
#P_p = 5.26 * 365  # precession period in days
#P_n = 354        # nutation period in days#

P_p = 1900 
P_n = 350

t_0 = time[np.argmax(flux_obs)]
t = time
y_obs = flux_obs
flux_err = flux_err  

omega_p = 2 * np.pi / P_p
omega_n = 2 * np.pi / P_n

# -----------------------------------
# Flux Model
# -----------------------------------
def flux_model_func(t, eta_0, Phi_o, OMEGA_p, OMEGA_n, omega_p, omega_n, Gamma, t0, F0):
    Theta_p_vals = omega_p * (t - t_0)
    Theta_n_vals = omega_n * (t - t_0)
    return model_func(Theta_p_vals, Theta_n_vals, OMEGA_p, OMEGA_n, eta_0, Phi_o, Gamma, F0)

# -----------------------------------
# lmfit Wrapper
# -----------------------------------
lm_model = lmfit.Model(flux_model_func)

params = lm_model.make_params(
    eta_0=np.deg2rad(15),
    Phi_o=np.deg2rad(9),
    OMEGA_p=np.deg2rad(16),
    OMEGA_n=np.deg2rad(5),
    omega_p=omega_p,
    omega_n=omega_n,
    Gamma=15.0,
    t0 = 59000, 
    F0=6
)

params['eta_0'].set(min=np.deg2rad(10), max=np.deg2rad(270))
params['Phi_o'].set(min=np.deg2rad(0), max=np.deg2rad(30))
params['OMEGA_p'].set(min=np.deg2rad(3), max=np.deg2rad(60))
params['OMEGA_n'].set(min=np.deg2rad(0.5), max=np.deg2rad(8))
params['Gamma'].set(min=1, max=30)
params['t0'].set(min=54952, max=60846)
params['F0'].set(min=1, max=10)

# Fit
result = lm_model.fit(y_obs, params, t=t)
print(result.fit_report())


plt.figure(figsize=(10, 5))
plt.plot(t, y_obs, 'o', label='Observed (with noise)', markersize=3)
plt.plot(t, result.best_fit, '-', label='Best Fit', linewidth=2)
#plt.plot(t, y_true, '--', label='True Flux (no noise)')
plt.xlabel("Time (days)")
plt.ylabel("Flux")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



# Best-fit parameters for MCMC
logf = -2
lmfit_para = np.array([
    result.best_values['eta_0'],
    result.best_values['Phi_o'],
    result.best_values['OMEGA_p'],
    result.best_values['OMEGA_n'],
    result.best_values['omega_p'],
    result.best_values['omega_n'],
    result.best_values['Gamma'],
    result.best_values['t0'],
    result.best_values['F0'],
    logf
])

# -----------------------------------
# Log-likelihood and Priors
# -----------------------------------
def log_prior(params):
    eta_0, Phi_o, OMEGA_p, OMEGA_n, omega_p, omega_n, Gamma, t0, F0, log_f = params
    if (np.deg2rad(10) < eta_0 < np.deg2rad(270) and
        np.deg2rad(0) < Phi_o < np.deg2rad(30) and
        np.deg2rad(3) < OMEGA_p < np.deg2rad(60) and
        np.deg2rad(0.5) < OMEGA_n < np.deg2rad(8) and
        (2*np.pi/1920) < omega_p < (2*np.pi/1800) and
        (2*np.pi/380) < omega_n < (2*np.pi/330) and
        3 < Gamma < 30 and
        1 < F0 < 10 and
        54952 < t0 < 60846 and
        -10 < log_f < 1):
        return 0.0
    return -np.inf

def log_likelihood(params, t, flux, flux_err):
    eta_0, Phi_o, OMEGA_p, OMEGA_n, omega_p, omega_n, Gamma, t0, F0, log_f = params
    model = flux_model_func(t, eta_0, Phi_o, OMEGA_p, OMEGA_n, omega_p, omega_n, Gamma, t0, F0)
    sigma2 = flux_err**2 + model**2 * np.exp(2 * log_f)
    return -0.5 * np.sum((flux - model)**2 / sigma2 + np.log(sigma2))

def log_posterior(params, t, flux, flux_err):
    lp = log_prior(params)
    return lp + log_likelihood(params, t, flux, flux_err) if np.isfinite(lp) else -np.inf

def chi2(params, t, y, yerr):
    return -2 * log_posterior(params, t, y, yerr)

# -----------------------------------
# MCMC Setup and Run
# -----------------------------------
ndim, nwalkers = 10, 32

def generate_initial_pos(center, nwalkers, scale=1e-8):
    pos = []
    while len(pos) < nwalkers:
        trial = center + scale * np.random.randn(ndim)
        if np.isfinite(log_posterior(trial, t, y_obs, flux_err)):
            pos.append(trial)
    return np.array(pos)

pos = generate_initial_pos(lmfit_para, nwalkers)

sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_posterior, args=(t, y_obs, flux_err)
)

print("Running MCMC...")
sampler.run_mcmc(pos, 10000, progress=True)
print("MCMC finished")


In [ ]:
burnin = 3000
thin = 10
flat_samples = sampler.get_chain(discard=burnin, thin=thin, flat=True)
flat_samples = flat_samples[np.isfinite(flat_samples).all(axis=1)]


np.savetxt('Corner_jet_precession_nutation_only_PKS1424-41.txt', flat_samples)
Sam = np.column_stack((np.rad2deg(flat_samples[:,0]),
                       np.rad2deg(flat_samples[:,1]),
                       np.rad2deg(flat_samples[:,2]),
                       np.rad2deg(flat_samples[:,3]),
                       2*np.pi / flat_samples[:,4],
                       2*np.pi / flat_samples[:,5],
                       flat_samples[:,6],
                       flat_samples[:,7],
                       flat_samples[:,8],
                       flat_samples[:,9]))

import corner
if Sam.shape[0] > 90:
    fig = corner.corner(Sam,
                    bins=20,
                   labels = [r"$\eta_0$", r"$\phi_0$", r"$\Omega_p$", r"$\Omega_n$", r"$T_p$", r"$T_n$", r"$\gamma$", r"$t_0$", r"$F_0$", r"$\log f$"],
                   color='blue',
                   quantiles=[0.16,0.5,0.84],
                   plot_contours=True,
                   fill_contours=True,
                   levels=(0.68,0.95,0.99),
                   plot_datapoints=True,
                   smooth=True,
                   smoothld=True,
                   title_fmt='.4g',
                   show_titles=True,
                   divergences=True, 
                   label_kwargs={"fontsize": 16},
                    title_kwargs={"fontsize": 14})
else:
    print(" Too few valid samples for corner plot")
plt.show()